# Annotation automatique : Le comte de Monte-Cristo
Dans le cadre de notre projet, nous avons décidé d'annoter un extrait de l'oeuvre célèbre de Dumas, *Le comte de Monte-Cristo* (1846). L'objectif est d'évaluer deux annotateurs automatiques de notre choix : on évaluera ici le module de Python, Spacy, ainsi que les annotateurs SEM et Treetagger.


In [31]:
# import modules
import spacy
import csv
import pandas as pd

from spacy.lang.fr import French
from spacy.tokens import Doc
from spacy.tokens import Span
from spacy import displacy

## Résultats
Afin de calculer les résultats qui nous intéressaient (c'est-à-dire, la précision, le rappel, et la f-mesure), nous avons implémenté trois fonctions : 
- `conf_matrix` : prend en argument les tokens de l'annotateur automatiques, les tokens du gold, `cat_s` qui est égale à la chaîne de caractère correspondant à la catégorie évaluée (par ex.: *POS* ou *lemma*), ainsi que les résultats de l'annotateur et du gold pour cette annotation. Dans le cas où on veut obtenir la matrice de confusion pour le *POS-tagging*, on appelera la fonction ainsi : `conf_matrix(token, token_gold, "POS", pos, pos_gold)`. La fonction permet un affichage des données, ce qui permet à l'utilisateur un aperçu visuel des différences entre les deux annotateurs. Le code prend en compte la différence de tokenisation entre l'annotateur automatique et le gold. Pour plus d'explication, voir le rapport. La fonction renvoie les vrai positifs, faux positifs, et faux négatifs.
- `conf_matrix_token` : même principe que `conf_matrix`, à la différence qu'on prend en argument seulement ici les tokens de l'annotateur automatique et du gold. Mets en avant la différence de tokenisation. La fonction renvoie les vrai positifs, faux positifs, et faux négatifs. 
- `p_r_fscore` : prend en arguement vrai positif, faux positif, et faux négatif, et renvoie les valeurs de précision, rappel, et f-mesure. 

In [32]:
# pos et lemma
def conf_matrix(token, token_gold, cat_s, cat, cat_gold):
    vp = 0
    fp = 0
    fn = 0

    for index, word in enumerate(pos):
        if token[index] == token_gold[index] and word == pos_gold[index]:
            vp += 1
        else:
            if cat_gold.isnull()[index] or word != cat_gold[index] and cat.notnull()[index]:
                fp += 1 
            elif cat.isnull()[index] or word != cat_gold[index] and cat_gold.notnull()[index]:
                fn += 1
            print(f'Token : {token[index]} | Token Gold : {token_gold[index]} | {cat_s} : {cat[index]} | {cat_s} Gold : {cat_gold[index]}')
    return vp, fp, fn

In [33]:
# token
def conf_matrix_token(token, token_gold):
    vp = 0
    fp = 0
    fn = 0

    for index, word in enumerate(token):
        if token[index] == token_gold[index]:
            vp += 1
        else:
            #when the tokens were divised not properly i was adding an empty row so we can compare row by row
            if token_gold.isnull()[index] or word != token_gold[index] and token.notnull()[index]:
                fp += 1 
            elif token.isnull()[index] or word != token_gold[index] and token_gold.notnull()[index]:
                fn += 1
            print(f'Token : {token[index]} | Token Gold : {token_gold[index]}')
    return vp, fp, fn

In [34]:
# calcul de r, p, et fscore
def p_r_fscore(vp, fp, fn):
    p = vp / (vp + fp)
    r = vp / (vp + fn)
    f = 2 * (p * r) / (p + r)
    return p, r, f

## Annotation automatique avec Spacy


In [35]:
# french language model
nlp = spacy.load("fr_core_news_sm")
# read text file & preprocess
with open('monte_cristo.txt','r') as file:
    txt = file.read()
stripTxt = txt.replace('\n', ' ')

In [36]:
# apply language model to preprocessed text 
doc = nlp(stripTxt)

In [37]:
print(f"{'text':{8}} {'POS':{6}}{'lemma':{6}}{'TAG':{6}} {'Dep':{6}} {'POS explained':{20}} {'tag explained'} ")
for token in doc:
    print(f'{token.text:{8}} {token.pos_:{6}} {token.lemma_:{6}} {token.tag_:{6}} {token.dep_:{6}} {spacy.explain(token.pos_):{20}} {spacy.explain(token.tag_)}')

text     POS   lemma TAG    Dep    POS explained        tag explained 
﻿Dantès  PUNCT  ﻿dantès PUNCT  nsubj  punctuation          punctuation
resta    PRON   rester PRON   nmod   pronoun              pronoun
confondu VERB   confondre VERB   ROOT   verb                 verb
:        PUNCT  :      PUNCT  punct  punctuation          punctuation
c'       PRON   ce     PRON   expl:subj pronoun              pronoun
était    AUX    être   AUX    cop    auxiliary            auxiliary
,        PUNCT  ,      PUNCT  punct  punctuation          punctuation
en       ADP    en     ADP    case   adposition           adposition
effet    NOUN   effet  NOUN   obl:mod noun                 noun
,        PUNCT  ,      PUNCT  punct  punctuation          punctuation
l'       DET    le     DET    det    determiner           determiner
explication NOUN   explication NOUN   conj   noun                 noun
de       ADP    de     ADP    case   adposition           adposition
ce       PRON   ce     PRON   nmod   

In [38]:
# creating csv file to store spacy output
# filename = "spacy_output.csv"
# header = ["TOKEN", "POS","LEMMA"]

#with open(filename, "w", newline="", encoding="UTF-8") as csvfile:
    #writer = csv.writer(csvfile)
    #writer.writerow(header)

    #for token in doc:
        #row = [token.text, token.pos_,token.lemma_]
        #writer.writerow(row)

### Évaluation de l'annotateur *Spacy* :
#### Rappel, précision, et f-score
Afin de manipuler les résultats obtenus avec Spacy (écrits dans un fichier .csv), on utilise le module *pandas*, importé précédemment.
On évalue ici deux annotations : 
- *POS tagging*
- *lemma* 

Ainsi, on va avoir deux matrices de confusions, une par annotation à évaluer.

In [39]:
# load csv files as dataframes
df_spacy = pd.read_csv('data/spacy_output.csv', error_bad_lines=False, sep = ',')
df_gold = pd.read_csv('data/spacy_output_gold.csv', error_bad_lines=False, sep = ';')

/tmp/ipykernel_26831/1734072599.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df_spacy = pd.read_csv('data/spacy_output.csv', error_bad_lines=False, sep = ',')
/tmp/ipykernel_26831/1734072599.py:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df_gold = pd.read_csv('data/spacy_output_gold.csv', error_bad_lines=False, sep = ';')


In [40]:
df_gold.head(8)

,TOKEN,POS,LEMMA
0,Dantès,NUM,Dantès
1,resta,VERB,rester
2,confondu,ADJ,confondre
3,:,PUNCT,:
4,c',PRON,ce
5,était,AUX,être
6,",",PUNCT,","
7,en,ADP,en


In [41]:
token = df_spacy['TOKEN']
pos = df_spacy['POS']
lemma = df_spacy['LEMMA']
token_gold = df_gold['TOKEN']
pos_gold = df_gold['POS']
lemma_gold = df_gold['LEMMA']

In [42]:
# confusion matrix
vp_pos, fp_pos, fn_pos = conf_matrix(token, token_gold, "POS", pos, pos_gold)
vp_lemma, fp_lemma, fn_lemma = conf_matrix(token, token_gold, "lemma", lemma, lemma_gold)
vp_token, fp_token, fn_token = conf_matrix_token(token, token_gold)

Token : ﻿Dantès | Token Gold : Dantès | POS : PUNCT | POS Gold : NUM
Token : resta | Token Gold : resta | POS : PRON | POS Gold : VERB
Token : confondu | Token Gold : confondu | POS : VERB | POS Gold : ADJ
Token : « | Token Gold : « | POS : NOUN | POS Gold : PROPN
Token : Faria | Token Gold : Faria | POS : ADJ | POS Gold : PROPN
Token : douze | Token Gold : douze | POS : VERB | POS Gold : NUM
Token : ans | Token Gold : ans | POS : ADJ | POS Gold : NOUN
Token : sont | Token Gold : sont | POS : AUX | POS Gold : VERB
Token : Vincennes | Token Gold : Vincennes | POS : NOUN | POS Gold : PROPN
Token : Dubuquoi | Token Gold : Dubuquoi | POS : ADV | POS Gold : ADJ
Token : Fort-l'Évêque | Token Gold : Fort-l'Évêque | POS : NOUN | POS Gold : PROPN
Token : celles | Token Gold : celles | POS : ADJ | POS Gold : PRON
Token : que | Token Gold : que | POS : SCONJ | POS Gold : PRON
Token : croyez | Token Gold : croyez | POS : NOUN | POS Gold : VERB
Token : profitons | Token Gold : profitons | POS : NOU

In [43]:
print(vp_pos, fn_pos, fp_pos)

1078 0 108


In [44]:
p_pos, r_pos, fscore_pos = p_r_fscore(vp_pos, fp_pos, fn_pos)
p_lemma, r_lemma, fscore_lemma = p_r_fscore(vp_lemma, fp_lemma, fn_lemma)
p_token, r_token, fscore_token = p_r_fscore(vp_token, fp_token, fn_token)

print(f'Pour annotation POS : P = {round(p_pos,3)} | R = {round(r_pos,3)} | f-score = {round(fscore_pos,3)}')
print(f'Pour annotation lemma : P = {round(p_lemma,3)} | R = {round(r_lemma,3)} | f-score = {round(fscore_lemma,3)}')
print(f'Pour tokenisation : P = {round(p_token,3)} | R = {round(r_token,3)} | f-score = {round(fscore_token,3)}')

Pour annotation POS : P = 0.909 | R = 1.0 | f-score = 0.952
Pour annotation lemma : P = 0.909 | R = 1.0 | f-score = 0.952
Pour tokenisation : P = 0.999 | R = 1.0 | f-score = 1.0


# TREETAGGER

In [45]:
# data importer
df_treetagger = pd.read_csv('data/treetagger_output.csv', error_bad_lines=False, sep = ';')
df_gold = pd.read_csv('data/treetagger_output_gold.csv', error_bad_lines=False, sep = ';')

# récupération des données de chaque colonne par le biais de l'affectation
token = df_treetagger['TOKEN']
pos = df_treetagger['POS']
lemma = df_treetagger['LEMMA']
token_gold = df_gold['TOKEN_GOLD']
pos_gold = df_gold['POS_GOLD']
lemma_gold = df_gold['LEMMA_GOLD']


/tmp/ipykernel_26831/1327330320.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df_treetagger = pd.read_csv('data/treetagger_output.csv', error_bad_lines=False, sep = ';')
/tmp/ipykernel_26831/1327330320.py:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df_gold = pd.read_csv('data/treetagger_output_gold.csv', error_bad_lines=False, sep = ';')


In [46]:
vp_token, fp_token, fn_token = conf_matrix_token(token, token_gold)

Token : --J'écrivais | Token Gold : --J'
Token : nan | Token Gold : écrivais
Token : - | Token Gold : nan
Token : dit-il | Token Gold : dit
Token : nan | Token Gold : -il
Token : quoiqu'il | Token Gold : Quoique
Token : nan | Token Gold : Il
Token : dit-il | Token Gold : dit
Token : nan | Token Gold : -il


In [47]:
vp_pos, fp_pos, fn_pos = conf_matrix(token, token_gold, "POS", pos, pos_gold)

Token : des | Token Gold : des | POS : PRP:det | POS Gold : DET:ART
Token : autres | Token Gold : autres | POS : ADJ | POS Gold : PRO:IND
Token : Faria | Token Gold : Faria | POS : NAM | POS Gold : NOM
Token : que | Token Gold : que | POS : PRO:REL | POS Gold : KON
Token : toutes | Token Gold : toutes | POS : PRO:IND | POS Gold : ADJ
Token : que | Token Gold : que | POS : KON | POS Gold : ADV
Token : -Vous | Token Gold : -Vous | POS : ADJ | POS Gold : PRO:PER
Token : Dantès | Token Gold : Dantès | POS : NAM | POS Gold : NOM
Token : soupirant | Token Gold : soupirant | POS : NOM | POS Gold : VER:ppre
Token : tous | Token Gold : tous | POS : PRO:IND | POS Gold : ADJ
Token : --Puis | Token Gold : --Puis | POS : ADJ | POS Gold : ADV
Token : qu' | Token Gold : qu' | POS : KON | POS Gold : ADV
Token : --Que | Token Gold : --Que | POS : ADJ | POS Gold : PRO
Token : --J'écrivais | Token Gold : --J' | POS : VER:impf | POS Gold : PRO:PER
Token : nan | Token Gold : écrivais | POS : nan | POS Gold

In [48]:
vp_lemma, fp_lemma, fn_lemma = conf_matrix(token, token_gold, "lemma", lemma, lemma_gold)

Token : des | Token Gold : des | lemma : du | lemma Gold : des
Token : autres | Token Gold : autres | lemma : autre | lemma Gold : autre
Token : Faria | Token Gold : Faria | lemma : Faria | lemma Gold : Faria
Token : que | Token Gold : que | lemma : que | lemma Gold : que
Token : toutes | Token Gold : toutes | lemma : tout | lemma Gold : tout
Token : que | Token Gold : que | lemma : que | lemma Gold : que
Token : -Vous | Token Gold : -Vous | lemma : -Vous | lemma Gold : -Vous
Token : Dantès | Token Gold : Dantès | lemma : Dantès | lemma Gold : Dantès
Token : soupirant | Token Gold : soupirant | lemma : soupirant | lemma Gold : soupirer
Token : tous | Token Gold : tous | lemma : tout | lemma Gold : tout
Token : --Puis | Token Gold : --Puis | lemma : --Puis | lemma Gold : --Puis
Token : qu' | Token Gold : qu' | lemma : que | lemma Gold : que
Token : --Que | Token Gold : --Que | lemma : --Que | lemma Gold : --Que
Token : --J'écrivais | Token Gold : --J' | lemma : --J'écrivais | lemma Gold

In [49]:
p_token, r_token, fscore_token = p_r_fscore(vp_token, fp_token, fn_token)
p_pos, r_pos, fscore_pos = p_r_fscore(vp_pos, fp_pos, fn_pos)
p_lemma, r_lemma, fscore_lemma = p_r_fscore(vp_lemma, fp_lemma, fn_lemma)

In [50]:
print(f'Pour annotation POS : P = {round(p_pos,5)} | R = {round(r_pos,5)} | f-score = {round(fscore_pos,5)}')
print(f'Pour annotation lemma : P = {round(p_lemma,5)} | R = {round(r_lemma,5)} | f-score = {round(fscore_lemma,5)}')
print(f'Pour tokenisation : P = {round(p_token,5)} | R = {round(r_token,5)} | f-score = {round(fscore_token,5)}')

Pour annotation POS : P = 0.93588 | R = 0.99631 | f-score = 0.96515
Pour annotation lemma : P = 0.93588 | R = 0.99631 | f-score = 0.96515
Pour tokenisation : P = 0.99567 | R = 0.99653 | f-score = 0.9961


# SEM

Puisque le logiciel ne génère que le résultat au format texte brut, nous avons donc dû établir un script permettant de le convertir en fichier CSV, ce qui rend adaptable au traitement dans le module PANDAS.

In [51]:
import csv

#with open('/content/sem_lzFne2.text','r',encoding = 'utf-8') as infile:
#  txt = infile.read()

#header = ["TOKEN", "POS"]

#with open('sem_output.csv', 'w', newline='') as outfile:
#    writer = csv.writer(outfile)
#    writer.writerow(header)
#    for line in txt.split(' '):
#        # on divise ici les deux attributs par l'espace
#        fields = line.strip().split('/')   
#        writer.writerow(fields)


In [52]:
sem_df = pd.read_csv('data/sem_output.csv', sep = ',')
gold_df = pd.read_csv('data/sem_output_gold.csv', sep = ',')

token = sem_df['TOKEN']
pos = sem_df['POS']
token_gold = gold_df['TOKEN_GOLD']
pos_gold = gold_df['POS_GOLD']


In [53]:
vp_token, fp_token, fn_token = conf_matrix_token(token, token_gold) 

Token : en_effet | Token Gold : en
Token : nan | Token Gold : effet
Token : il_y_a | Token Gold : il
Token : nan | Token Gold : y
Token : nan | Token Gold : a
Token : Et_puis | Token Gold : Et
Token : nan | Token Gold : puis
Token : Il_y_a | Token Gold : il
Token : nan | Token Gold : y
Token : nan | Token Gold : a
Token : celles-là | Token Gold : celles-
Token : nan | Token Gold : -là
Token : croyez-moi | Token Gold : Croyez
Token : nan | Token Gold : -moi
Token : profitons-en | Token Gold : Profitons
Token : nan | Token Gold : -en
Token : point_qu' | Token Gold : point
Token : nan | Token Gold : _qu'
Token : -J' | Token Gold : --J'
Token : -Non | Token Gold : --Non
Token : vous_faites_du | Token Gold : vous
Token : nan | Token Gold : faites
Token : nan | Token Gold : du
Token : Faria | Token Gold : 
Faria
Token : à_l'_ombre_du | Token Gold : à
Token : nan | Token Gold : l'
Token : nan | Token Gold : ombre
Token : nan | Token Gold : du
Token : au_pied_de | Token Gold : au
Token : nan |

In [54]:
vp_pos, fp_pos, fn_pos = conf_matrix(token, token_gold, "POS", pos, pos_gold)

Token : ﻿Dantès | Token Gold : ﻿Dantès | POS : NC | POS Gold : NPP
Token : en_effet | Token Gold : en | POS : ADV | POS Gold : P
Token : nan | Token Gold : effet | POS : nan | POS Gold : NC
Token : il_y_a | Token Gold : il | POS : P | POS Gold : CLS
Token : nan | Token Gold : y | POS : nan | POS Gold : CLO
Token : nan | Token Gold : a | POS : nan | POS Gold : V
Token : Et_puis | Token Gold : Et | POS : CC | POS Gold : CC
Token : nan | Token Gold : puis | POS : nan | POS Gold : CC
Token : continua | Token Gold : continua | POS : P | POS Gold : V
Token : Latude | Token Gold : Latude | POS : NC | POS Gold : NPP
Token : Il_y_a | Token Gold : il | POS : P | POS Gold : CLS
Token : nan | Token Gold : y | POS : nan | POS Gold : CLO
Token : nan | Token Gold : a | POS : nan | POS Gold : V
Token : celles-là | Token Gold : celles- | POS : PRO | POS Gold : PRO
Token : nan | Token Gold : -là | POS : nan | POS Gold : ADV
Token : attendons | Token Gold : attendons | POS : NC | POS Gold : VIMP
Token : 

In [55]:
p_token, r_token, fscore_token = p_r_fscore(vp_token, fp_token, fn_token)
p_pos, r_pos, fscore_pos = p_r_fscore(vp_pos, fp_pos, fn_pos)
p_lemma, r_lemma, fscore_lemma = p_r_fscore(vp_lemma, fp_lemma, fn_lemma)

In [56]:
print(f'Pour annotation POS : P = {round(p_pos,3)} | R = {round(r_pos,3)} | f-score = {round(fscore_pos,3)}')
print(f'Pour annotation lemma : P = {round(p_lemma,3)} | R = {round(r_lemma,3)} | f-score = {round(fscore_lemma,3)}')
print(f'Pour tokenisation : P = {round(p_token,3)} | R = {round(r_token,3)} | f-score = {round(fscore_token,3)}')

Pour annotation POS : P = 0.943 | R = 0.95 | f-score = 0.946
Pour annotation lemma : P = 0.936 | R = 0.996 | f-score = 0.965
Pour tokenisation : P = 0.968 | R = 0.951 | f-score = 0.959


In [57]:
print(f'Pour annotation POS : P = {round(p_pos,5)} | R = {round(r_pos,5)} | f-score = {round(fscore_pos,5)}')
print(f'Pour annotation lemma : P = {round(p_lemma,5)} | R = {round(r_lemma,5)} | f-score = {round(fscore_lemma,5)}')
print(f'Pour tokenisation : P = {round(p_token,5)} | R = {round(r_token,5)} | f-score = {round(fscore_token,5)}')

Pour annotation POS : P = 0.94288 | R = 0.94977 | f-score = 0.94631
Pour annotation lemma : P = 0.93588 | R = 0.99631 | f-score = 0.96515
Pour tokenisation : P = 0.9676 | R = 0.95133 | f-score = 0.95939
